### **Import Libraries**

Fact Table:
1 row per employee per course per fiscal period


In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
import pandas as pd
import numpy as np

### **IT Employee Database Dataset, Merging email with new**



In [ ]:
#Loading in all the files for data cleaning

#loading new employee dataset received from Stacey Cook, which originates from the SAP Hana Database
it_employee = pd.read_excel('/content/IT Org Hierarchy_4.12.xlsx')
#loading new successfactors dataset
employee_successfactors = pd.read_csv('/content/SuccessFactors_employee_4.12.csv')
#loading OLD employee data file from previous fiscal period
emp_old = pd.read_csv('/content/updated_employee_P7v2.csv')
#loading new Pluralsight data
content_usage = pd.read_csv('/content/courses_summary_P8.csv')
#loading new Linkedin Dataset
linkedin_lmanage = pd.read_csv('/content/LearnerDetail_P8.csv')
#loading new Microsoft dataset
microsoft = pd.read_excel('/content/Microsoft_P8.xlsx', sheet_name='Export')
#loading new SAP Learning hub dataset
sap_hub = pd.read_csv('/content/SAP_P8.csv')
#loading Successfactors dataset
successfactors = pd.read_csv('/content/UserLearningHoursCSV_P8.csv')
#updated employee database
it_list = pd.read_csv('/content/updated_employee_P7v2.csv')
#loading ad-hoc dataset
ad_hoc = pd.read_csv('/content/Ad-hoc_P8.csv')
#load previous fiscal period data for all the data sources(Pluralsight,Linkedin,Microsoft,SAP,SuccessFactors,ad-hoc)
prev_data = pd.read_csv('/content/data_total_All.csv')

In [ ]:
department_short = {
    55 : 'Info Tech',
    555 :	'IT Intl',
    600 :	'IT Warehouse',
    601 :	'IT Membership',
    602 :	'IT Ancillary',
    603 :	'IT Merchandise',
    604 :	'IT Depot & Mfg',
    605 :	'IT eCom Dev',
    606 :	'IT Corporate',
    608 :	'IT eCom Ops',
    609 :	'IT eComm Site Ops & Infra',
    621 :	'IT Info Sec',
    622 :	'IT Platform Ops',
    624 :	'IT Service Support',
    625 :	'IT Ent Archit',
    626 :	'IT Business Ops',
    627 :	'IT Compliance',
    631 :	'IT Org Dev',
    632 :	'IT Infrastructure',
    634 :	'IT Data Serv',
    635	: 'IT Shared Services',
    277	: 'Data Center Ops',
    840 :	'Ecomm SIP',
}

In [ ]:
#loading NEW employee dataset received from Stacey Cook
it_employee = pd.read_excel('/content/IT Org Hierarchy_4.12.xlsx')
#loading successfactors dataset
employee_successfactors = pd.read_csv('/content/SuccessFactors_employee_4.12.csv')

#DATA CLEANING SECTION

#making a dataframe of just the User ID and email address to merge it into employee database file to get their email addresses
emp_short = employee_successfactors[['User ID','Email Address']]

#lowercasing the email address, so it's not all caps
emp_short['Email Address'] = emp_short['Email Address'].str.lower()

#merge the dataframes, so that I have the email addresses of the employees
employee_merged = pd.merge(it_employee, emp_short, left_on='EMP_EMPLOYEE_NUM',right_on='User ID', how='left')

#rename the columns
employee_merged.rename(columns = {'EMP_EMPLOYEE_NUM' : 'Employee ID', 'EMP_NAME' : 'Official Name', 'EMP_DEPARTMENT_NUM' : 'Dept_Number' 
                                  , 'EMP_DEPARTMENT_DESC' : 'Department','EMP_JOB_DESC' : 'Job Title', 'Email Address' : 'Email'}, inplace=True)
#removing Costco Logistics from the dataset (They're not part of our division wide goal)
employee_merged = employee_merged.loc[employee_merged['Dept_Number']!=610]

#creating an empty column called Dept Short
employee_merged['Dept Short']=np.nan
#filling in the department short names
employee_merged['Dept Short'] = employee_merged['Dept Short'].fillna(employee_merged['Dept_Number'].map(department_short))

#removing the duplicate employees because each employee has a title 'TRAINEE' AND 'TRAINEE CLERK'
employee_merged = employee_merged.loc[employee_merged['Job Title']!='TRAINEE']

#capitalizing the first letter of each name in the official name column
employee_merged['Official Name'] =  employee_merged['Official Name'].str.title()

#making the column name for Dept# and Dept short together, which I use in power bi
employee_merged['Dept#/Name'] = employee_merged['Dept_Number'].astype(str) + ' ' + employee_merged['Dept Short']

#adding in george mcallister because the employee database file leaves him out for some reason and I need him for the org hierarchy slicer
george = [{'COMPMANY_CD' : 1, 'Employee ID' : 6341466, 'Official Name' : 'George Mcallister', 'Dept_Number' : 632, 'Department' : 'Infrastructure', 'Job Title' : 'NETWORK SUPPORT IT SUPERVISOR',
           'MGR_NAME' : 'JASON ALLEN HORN', 'Email' : 'gmcallister@costco.com','Dept Short' : 'IT Infrastructure', 'Dept#/Name' : '632 IT Infrastructure'}]
george = pd.DataFrame(george)
employee_merged = pd.concat([employee_merged, george])

/usr/local/lib/python3.7/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [ ]:
#creating org hierarchy into 1 column called ParentKey
employee_merged['ParentKey'] = employee_merged['MGR_NAME'].fillna(it_employee['DIR_NAME'])
employee_merged['ParentKey'] = employee_merged['ParentKey'].fillna(it_employee['AVP_NAME'])
employee_merged['ParentKey'] = employee_merged['ParentKey'].fillna(it_employee['VP_NAME'])
employee_merged['ParentKey'] = employee_merged['ParentKey'].fillna(it_employee['REPORTING_TO'])

#creates dictionary of the employee official name and the employee ID #
employee_dict = dict(zip(employee_merged['Official Name'],employee_merged['Employee ID']))

#changing the columng ParentKey names with the title, so it can match it to the official names in the dictionary
employee_merged['ParentKey'] = employee_merged['ParentKey'].str.title()
#changing the column ParentKey from names to employee ID #s
employee_merged['ParentKey'] = employee_merged['ParentKey'].replace(employee_dict, regex=True)
#Removing the cu@costco.com email address that is a incorrect placeholder in the SuccessFactor database
employee_merged['Email'].mask(employee_merged['Email'] == 'cu@costco.com', np.nan, inplace=True)

#changes parentkey value from Richard Galanti to being under Terry Williams, now Katie Hall will be under Terry Williams
employee_merged.loc[employee_merged['Employee ID']==442253, 'ParentKey'] = 229414
#removes richard galanti as parentkey, which just refers to the employee Terry williams and removes his ParentKey value, so he is on top of the org hierarchy chart
employee_merged.loc[employee_merged['Employee ID']==229414, 'ParentKey'] = np.nan

In [ ]:
#james b husted and ahmed a osmani have empty email addresses need to fill in

#need to fill in empty email addressses

In [ ]:
employee_merged = employee_merged.drop_duplicates()

In [ ]:
employee_merged[employee_merged.duplicated(['Email'], keep=False)]

,COMPMANY_CD,Employee ID,Official Name,Dept_Number,Department,EMP_JOB_CD,Job Title,MGR_NAME,DIR_NAME,AVP_NAME,VP_NAME,REPORTING_TO,User ID,Email,Dept Short,Dept#/Name,ParentKey,Display Name,Title Short
966,1,609656,James B Husted,277,Data Center,5505.0,COMPUTER OPERATOR,NaN,NaN,NaN,NaN,AMY BROWN,NaN,NaN,Data Center Ops,277 Data Center Ops,229282.0,NaN,Emp
1233,1,6177309,Ahmed A Osmani,622,IS EA Pltfrms,5907.0,ENGINEER 2,SUMIT KAPUR,BHAWANDEEP S KAILA,SREEPRAVEEN ARUGUNTA,PATRICIA BAUER,NaN,NaN,NaN,IT Platform Ops,622 IT Platform Ops,6151964.0,NaN,Emp


In [ ]:
#Using old employee database file to get the "Display Name" and "Email" that is missing with the new dataset
emp_old = pd.read_csv('/content/updated_employee_P7v2.csv')
emp_old_filtered = emp_old[['Employee ID','Display Name','Email']]
employee_merged = pd.merge(employee_merged, emp_old_filtered, left_on='Employee ID', right_on='Employee ID', how='inner')
employee_merged['Email_x'] = employee_merged['Email_x'].fillna(employee_merged['Email_y'])
employee_merged.rename(columns={'Email_x':'Email'},inplace=True)
employee_merged.drop({'Email_y'},axis=1,inplace=True)

In [ ]:
#code block is to solve the issue with employees in Canada IT having a ParentKey with Yoram Bernard Rubanenko, but that employee isn't their boss

#deleting the ParentKey that states Yoram Bernard Rubanenko
employee_merged['ParentKey'].mask(employee_merged['ParentKey'] == 'Yoram Bernard Rubanenko', np.nan, inplace=True)
#creating a small dataframe from the previous fiscal period with employee ID and parentkey to merge into the new employee dataset
emp_old_parentkey = emp_old[['Employee ID','ParentKey']]
#merging the 2 dataframes together
employee_merged = pd.merge(employee_merged, emp_old_parentkey, left_on='Employee ID', right_on='Employee ID', how='inner')
#filling in the missing parentkey values
employee_merged['ParentKey_x'] = employee_merged['ParentKey_x'].fillna(employee_merged['ParentKey_y'])
#renaming the column to match the previous column name
employee_merged.rename(columns={'ParentKey_x':'ParentKey'},inplace=True)
#removing the column that I don't need
employee_merged.drop({'ParentKey_y'},axis=1,inplace=True)

In [ ]:
#Adding a column called Title Short that ouputs their title, IE SVP for Senior Vice President etc., This column is used in the Power BI Org Slicer
employee_merged.loc[employee_merged['Job Title']=='SR VICE PRESIDENT', 'Title Short'] = 'SVP'
employee_merged.loc[employee_merged['Job Title']=='VICE PRESIDENT', 'Title Short'] = 'VP'
employee_merged.loc[employee_merged['Job Title']=='ASST VICE PRESIDENT', 'Title Short'] = 'AVP'
employee_merged.loc[employee_merged['Job Title']=='DIRECTOR', 'Title Short'] = 'Dir'
employee_merged.loc[employee_merged['Job Title']=='MANAGER', 'Title Short'] = 'Mgr'
employee_merged.loc[employee_merged['Job Title']=='IT SUPERVISOR', 'Title Short'] = 'Sup'
employee_merged.loc[employee_merged['Job Title']=='NETWORK SUPPORT IT SUPERVISOR', 'Title Short'] = 'Sup'
employee_merged.loc[employee_merged['Title Short'].isna(), 'Title Short'] = 'Emp'

In [ ]:
employee_merged.to_csv('employee_dataset.csv', index=False)

In [ ]:
employee_merged.isna().sum()

COMPMANY_CD         0
Employee ID         0
Official Name       0
Dept_Number         0
Department          0
EMP_JOB_CD          1
Job Title           0
MGR_NAME          594
DIR_NAME          446
AVP_NAME          454
VP_NAME           353
REPORTING_TO     2117
User ID            65
Email               2
Dept Short          0
Dept#/Name          0
ParentKey           2
Display Name       41
Title Short         0
dtype: int64

### **Pluralsight Dataset**



In [ ]:
#new dataset TESTING
content_usage = pd.read_csv('/content/courses_summary_P8.csv')

In [ ]:
#not merging the employee data
#Changing the data cleaning

#filtering out the data before fiscal year 2022
content_usage = content_usage.loc[content_usage['LastViewDate']>'2021-08-29']

#making sure the emails are all lowercase
content_usage['Email'] = content_usage['Email'].str.lower()
#removing columns from the pluralsight dataset
content_usage.drop(['FirstName','LastName', 'AccountName','TeamName'], axis=1, inplace=True)
#labeling the dataset as Pluralsight
content_usage['Platform'] = 'Pluralsight'
#removing the data that has a nan value under total hours, some courses have a NaN value for some reason.
#Another cause is the exam and lab don't have an hour time associated with it.
content_usage = content_usage[content_usage['TotalHours'].notna()]

In [ ]:
content_usage['Period'] = 8

### **Linkedin Learning Dataset**

In [ ]:
#Linkedin Dataset
linkedin_lmanage = pd.read_csv('/content/LearnerDetail_P8.csv')

In [ ]:
#changing the data cleaning

#labeling the data source as Linkedin
linkedin_lmanage.drop(['Name', 'Unique User ID','Groups (Current Membership)'], axis=1, inplace=True)
linkedin_lmanage.rename(columns = {'Last Viewed During Time Range (PST/PDT)' : 'LastViewDate', 'Content Type' : 'ContentType',
                                   'Content ID' : 'ContentId', 'Content Name' : 'ContentName', 'Percent Completed' : 'CompletionPercentage',
                                   'Hours Viewed (LinkedIn Learning and Organization Content Only)' : 'TotalHours'}, inplace= True)
#add column Platform with the word Linkedin added into it
linkedin_lmanage['Platform'] = 'Linkedin'
#removing columns that are not necessary
linkedin_lmanage.drop('Content Provider', axis=1, inplace=True)

#rows that have data without null values in the Hours Viewed column
linkedinl_manage = linkedin_lmanage.dropna(subset=['TotalHours'])

In [ ]:
linkedinl_manage['Period'] = 8

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


### Microsoft Learning Dataset

In [ ]:
microsoft = pd.read_excel('/content/Microsoft_P8.xlsx', sheet_name='Export')

/usr/local/lib/python3.7/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
#dictionary created with the course title and the hours required to completed the course
course_hours = { 'AZ-140' : 32,'AZ-104-EP' : 4, 'AZ-104' : 32,
    'AZ-204' : 40, 'AZ-204-EP' : 4,
    'AZ-304' : 32, 'AZ-303' : 40,'AZ-303-EP' : 4, 'AZ-304-EP' : 4, 'AZ-305' : 32,
    'AZ-400' : 40, 'AZ-400-EP' : 4,
    'AZ-500' : 32, 'AZ-500-EP' : 4,
    'AZ-700' : 24,
    'AZ-900' : 16, 'AZ-900-2day' : 16, 'FY21:MATD:AZ-900' : 16,
    'DP-090' : 8,
    'DP-100' : 24, 'DP-100-EP' : 4,
    'DP-203': 32, 'DP-203-EP' : 4,
    'DP-300' : 32, 'DP-300-EP' : 4,
    'DP-900' : 16, 'DP-900-EP' : 4,  'FY21:MATD: DP-900' : 16,
    'DA-100' : 32, 'DA-100-EP' : 4,
    'SC-200-EP' : 4,
    'SC-400-EP' : 4,
    'SC-900' : 16, 'SC-900-EP' : 4, 'FY22: Security: SC-900' : 16,
    'PL-901' : 8,
    'AI-102-EP': 4,
    'AI-900' : 8, 'AI-900-EP' : 4, 'FY21:MATD: AI-900' : 8,
    'OH-001' : 24, 'OH-004': 24,'OH-006' : 24,
    'FY21: Security: Secure Organization Data' : 16, 'FY21:MATD: GitHub' : 16, 'FY21:MATD: Migrating-On-Premises' : 16,
    'FY21: BizApp: App Maker' : 8, 'FY21:BizApp:PL-901' : 8,
    'FY21: MATD: Synapse' : 16,
    'FY22: Security: Modernize Security & Defend Threats' : 6,
    'MS-500-EP' : 4,
    }

In [ ]:
#Data Cleaning

#removing the last rows that include the totals
microsoft = microsoft[:-2]

#DATES ARE HARDCODED, NEED TO MANUALLY CHANGE THE DATE TO PULL THE SPECIFIC FISCAL PERIOD!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
microsoft = microsoft.loc[microsoft['Delivery Date']>'2022-03-13']
microsoft = microsoft.loc[microsoft['Delivery Date']<'2022-04-11']


#replacing these specific columns with 0 instead of Null values
microsoft.loc[:,('Registered','No-Shows','Canceled','Attended','Wait-Listed')]= microsoft.loc[:,('Registered','No-Shows','Canceled','Attended','Wait-Listed')].fillna(0)
#remove scheduled column, because it's an empty column
microsoft.drop(['Registered Domain','Delivery Status'], axis=1, inplace=True)
#copying the Course Code column into the TotalHours column
microsoft['TotalHours']=microsoft['Course Code'].copy()
#if they attended the class(value of 1) I will map the course hours dictionary with the course code and corresponding hours associated with the class
microsoft['TotalHours']= microsoft.loc[microsoft['Attended']==1, 'TotalHours'].map(course_hours)
#filling in the nan values in TotalHours with 0s
microsoft['TotalHours'] = microsoft['TotalHours'].fillna(0)
#creating column with the value Microsoft to denote the original data source
microsoft['Platform']= 'Microsoft'
#creating column with the content type of instructor led
microsoft['ContentType']= 'Instructor-Led Course'
#renaming the columns
microsoft.rename(columns = {'Delivery ID' : 'ContentId', 'Catalog Name' : 'ContentName','Learner Work Email' : 'Email',
                            'Delivery Date' : 'LastViewDate'}, inplace= True)

In [ ]:
microsoft['LastViewDate']= pd.to_datetime(microsoft['LastViewDate'])
microsoft['LastViewDate'] = microsoft.LastViewDate.dt.date

d= {'1' : [('2021-08-30', '2021-09-26')],
  '2' : [('2021-09-27', '2021-10-24')],
  '3' : [('2021-10-25', '2021-11-21')],
  '4' : [('2021-11-22', '2021-12-19')],
  '5' : [('2021-12-20', '2022-01-16')],
  '6' : [('2022-01-17', '2022-02-13')],
  '7' : [('2022-02-14', '2022-03-13')],
  '8' : [('2022-03-14', '2022-04-10')],
  }

#What should I call the date
#microsoft['LastViewDate'] = pd.to_datetime(microsoft['LastViewDate'], format='%m/%d/%Y')
microsoft['LastViewDate'] = pd.to_datetime(microsoft['LastViewDate'], format='%Y/%m/%d')

for k, v in d.items():
    for s, e in v:
        microsoft.loc[microsoft['LastViewDate'].between(s,e), 'Period'] = k

Checking microsoft courses and hours total per period

### **SAP Learning Hub**

In [ ]:
sap_hub = pd.read_csv('/content/SAP_P8.csv')

In [ ]:
#removing the data with a null value for time spent
sap_hub = sap_hub[sap_hub['Time Spent (hh:mm:ss)'].notna()]
#changing the time spent from hh:mm:ss to hours in decimals
sap_hub['Time Spent (hh:mm:ss)'] =   pd.to_timedelta(sap_hub['Time Spent (hh:mm:ss)']).dt.total_seconds()/3600
#removing unnecessary columns
sap_hub.drop(['User ID','First Name','Last Name','Country','Primary Supervisor','Custom 1','Custom 2','Custom 3',
              'Item Type','Object','Object Last Activity','Object Completed','Times Accessed','Subject Area 1',
              'Subject Area 2','Subject Area 3'], axis=1, inplace=True)
#rename columns
sap_hub.rename(columns={'Email Address' : 'Email','Item ID' : 'ContentId','Item Estimated Duration in hours': 'ContentDuration',
                        'Item Title': 'ContentName','Subject Area':'Subjects','Time Spent (hh:mm:ss)' : 'TotalHours',
                        'User Last Activity':'LastViewDate'}, inplace = True)
sap_hub['Platform']='SAP'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-co

In [ ]:
sap_hub['Period'] = 8

### **SuccessFactors Dataset**

In [ ]:
#updated employee database
it_list = pd.read_csv('/content/updated_employee_P7v2.csv')

In [ ]:
successfactors = pd.read_csv('/content/UserLearningHoursCSV_P8.csv')

In [ ]:
#Success Factors Dictionary for classes with no hours associated
success_hours = {'101 - The Art and Science of Writing User Stories' : 1.5, 
        '201 - The Art and Science of Writing Technical User Stories' : 1.5, 
        'Accessibility Fundamentals: Disabilities, Guidelines, and Laws' : 0.5,
        'Activation Learning Module 1 - Operating Model Overview' : 0.15,
        'Activation Learning Module 2 - Product vs. Project Teams' : 0.15,
        'Activation Learning Module 3 - Product Overview' : 0.15,
        'Activation Learning Module 4 - Delivery Pods' : 0.15,
        'Activation Learning Module 5 - Interaction Model' : 0.15,
        'Activation Learning Module 6 - DevOps 101' : 0.15,
        'Aerosol Can Puncture Device Training Acknowledgement' : 0.05,
        'Agile 101 Module 1: Intro to DevOps' : 0.2,
        'Agile 101 Module 10: Minimum Viable Product' : 0.2,
        'Agile 101 Module 2: What is Agile' : 0.2,
        'Agile 101 Module 3: ScrumXP Overview' : 0.2,
        'Agile 101 Module 4: ScrumXP Roles' : 0.2,
        'Agile 101 Module 5: ScrumXP Ceremonies' : 0.2,
        'Agile 101 Module 6: Backlog Refinement' : 0.2,
        'Agile 101 Module 7: Kanban Introduction' : 0.2,
        'Agile 101 Module 8: User Stories' : 0.2,
        'Agile 101 Module 9: Measuring Progress' : 0.2,
        'Agile Leadership Training' : 18,
        'All - LOTO/Elec Awareness (P4)' : 0.19,
        'ALLY: The Neuroscience of Advocating for Others' : 3,
        'An Introduction to OKRs' : 0.2,
        'Asbestos Awareness Training - Logistics' : 0.15,
        'Bloodborne Pathogens (BBP) Quiz and Acknowledgment' : 0.1,
        'Building 7 Emergency Procedures' : 0.25,
        'Career Development and Continuous Performance in SuccessFactors' :	1.5,
        'CARTS Training'	:	0.5,
        'Clean Desk' :	0.25,
        'Confidentiality Statements for IT Payroll and Accounting Teams' : 	0.02,
        'Confined Spaces P5/P6 - Optical' :	0.5,
        'Costco Ergonomics Awareness Training' :	0.25,
        'Costco Hazardous Materials DOT Training Course: Part 1' :	0.42,
        'Costco Hazardous Materials DOT Training Course: Part 2' :	0.56,
        'Costco Pay Policies' : 0.5,
        'Custom JavaScript/ARIA Widgets' :	0.5,
        'Crucial Conversations for Accountability' : 12.5,
        'Department Specific / JHA for Logistics Locations' : 0.15,
        'Designing an Accessible User Experience' : 0.5,
        'Device-Independent User Input Methods' :	0.5,
        'Drawing the Line (US) - Modules 01 to 03' :	1.5,
        'Drawing the Line (US) - Modules 04 to 06' :	1.5,
        'Drawing the Line (US) - Modules 07 to 09' :	1.5,
        'Drawing the Line (US) - Modules 10 to 12' :	1.5,
        'Dynamic Updates, AJAX, Single Page Apps' :	0.5,
        'Emergency Procedures (P5-P6)' : 0.25,
        'Everyday Bias Training' : 1,
        'Fire Extinguishers Quiz and Acknowledgment' :	0.19,
        'First Aid (All) Quiz and Acknowledgment' : 0.1,
        'Form Labels, Instructions, and Validation' :	0.5,
        'General Data Protection Regulation (GDPR) Overview for IT Employees' :	0.5,
        'Getting Things Done (GTD)' : 8,
        'HazMat RTV Bucket Program' :	0.19,
        'High Profile High Demand Items Acknowledgment' :	0.1,
        'HIPAA Basics (non-healthcenter employees)' :	0.5,
        'HIPAA Mobile Device Procedure & Acknowledgement' :	0.08,
        'Hire for Success (USA)' :	0.4,
        'Home & Regional Office CCPA Awareness' :	0.05,
        'Incident Analysis Training for Logistics Locations' : 0.15,
        'Influencer Training' : 7.5,
        'Images, SVG, and Canvas' :	0.5,
        'Introduction to Concur Travel & Expense'	: 0.5,
        'IT New Team Member Warehouse Tour' : 1.5,
        'IT User Security Agreement' :	0.1,
        'ITR Operating Model - Leading Change Module' :	0.6,
        'Leadership Development 101 Acknowledgment' :	32,
        'Leadership Development and Management Expectations' :	0.3,
        'Legal: Document Retention Notice' :	0,
        'Lifting/Ergo (P5-P6)' :	0.19,
        'Logistics Locations Confined Spaces Quiz and Acknowledgment' : 0.15,
        'Marketing/Membership Policies & Procedures' :	0.17,
        'Non Foods Quality Assurance Product Safety' :	0.6,
        'PCI and Card Security for IT and Corporate Users - Annual Training' :	0.19,
        'PCI and Card Security for Non Corporate Users - Annual Training' :	0.19,
        'Pedestrian Safety Training for Logistics Locations' : 0.15,
        'Pedestrian Safety - Optical Lab (P5_P6)' :	0.15,
        'Period 7 Fall Protection Quiz Acknowledgement' : 0.1,
        'Phase 1 Learning Completion' :	0.1,
        'Phishing Annual Training' :	0.19,
        'PPE / Ladders Quiz and Acknowledgment' : 0.15,
        'React Training' : 40,
        'Responsive Design and Zoom' :	0.5,
        'SAFe For Teams' : 16,
        'SAFe Scrum Master Training' : 13,
        'Safety Program Overview Quiz and Acknowledgement Statement for Logistics Locations' : 0.15,
        'Security Essentials for Corporate Users - Annual Training' :	0.19,
        'Semantic Structure and Navigation' :	0.5,
        'Spill Prevention Control and Countermeasures Plan (SPCC) Training' :	0.25,
        'Standard of Ethics for Managers / Supervisors March 2021' :	0.1,
        'Statement of Confidentiality Acknowledgment' :	0,
        'Supervisors Orientation Online Acknowledgment' :	0.1,
        'Tactical Leadership: Coaching Skills Development Program' : 6,
        'TEST NOT YET RELEASED Intro to Engine Components' :	0,
        'TEST NOT YET RELEASED The Importance of Quality Fuel on Engine Performance' :	0,
        'The Scrum Master as Servant Leader' :	2,
        'THINK Again Quiz and Acknowledgment' :	0.1,
        'THINK Lifting Quiz and Acknowledgment Statement - Logistics' : 0.15,
        'Timekeeping Policy Acknowledgement' :	0.05,
        'Trailer Unloading and Loading Quiz and Acknowledgment' : 0.15,
        'Universal Waste Training (short version)' :	0.2,
        'Using Option STOP' :	0.33,
        'Visual Design and Colors' :	0.19,
        'Web Accessibility Testing: Basic Methods and Tools' :	0.5,
        'Web Accessibility Testing: Screen Readers' :	0.5,
        'Web Accessibility Testing: WCAG Conformance Testing, Detailed Methodology' :	0.5,
        'Workplace Violence Quiz and Acknowledgment - Logistics' : 0.15,
        'Writing Technical User Stories' : 2,
        'Writing User Stories' : 2,

        #new FY22 P8 classes with 0 hours
        'Tactical Leadership: Performance Improvement' : 1.5,
        'From Unconscious Bias to Conscious Inclusion - People Leader Training' : 1.5,
        'Tactical Leadership: Budgeting, Financials and CARTS+' : 4,
        'Leading SAFe' : 16,
        'Crucial Conversations for Accountability' : 12.5,
        'SAFe Release Train Engineer (aka: Delivery Manager)' : 24,
        'OWASP Fundamentals for Developers' : 0.15,
        'Leadership Development 101 Acknowledgement' : 32,
        'An Introduction to OKRs' : 0.2,

        'Phishing Annual Training' : 0.19,
        'PCI and Card Security for IT and Corporate Users - Annual Training' : 0.19,
        'Clean Desk' : 0.19,
        'Destruction of Sensitive Information or Assests' : 0.19,

        'Privacy Essentials For IT & Home Office Employees' : 0.5, #TEMPORARY CODE
        
}

In [ ]:
#code is emptying privacy essentials for it & home office employees training for some odd reason, original dataset has 0.5
#maybe the code replacing the 0 with nans is screwing up the system

In [ ]:
#merging the canadian and usa data source together
#frame_success = [success_fac_usa, success_fac_can]
success_fac = successfactors
#success_fac = pd.concat(frame_success)

success_fac['Contact Hours'] = success_fac['Contact Hours'].replace(0, np.nan, inplace=True)
#based on the success hours dictionary adding the # of hours for each class not already listed in the system
success_fac['Contact Hours'] = success_fac['Contact Hours'].fillna(success_fac['Item Title'].map(success_hours))
#creating a column called platform that says Costco U to denote the data source
success_fac['Platform'] = 'Costco U'
#making copy of employee database to merge in the email to use as a primary key to merge success factors with the other data sources
it_list2 =it_list[['Employee ID','Email']].copy()
#renaming the User ID column, so I have a common column to merge on
success_fac.rename(columns = {'User ID' : 'Employee ID','Completion Date' : 'LastViewDate', 'Item Type' : 'ContentType',
                              'Item ID' : 'ContentId', 'Item Title' : 'ContentName','Contact Hours' : 'TotalHours'}, inplace= True)
#removing the "Pacific Time" from the LastViewDate column
success_fac['LastViewDate']  = success_fac['LastViewDate'].str[:-13]
#merging the employee email address into successfactor data, so I can use email as a primary key to merge the data sources
data_success = it_list2.merge(success_fac, left_on='Employee ID', right_on='Employee ID', how='right')
#removing unnecessary columns
data_success.drop(['Employee ID','Active User','First Name','Last Name','Middle Name','Manager ID', 'Manager First Name' ,'Manager Last Name',
                   'Manager Middle Name','Department ID', 'Organization Description','Job Code ID','Job Location ID','Job Location Description',
                   'Employee Type ID','Employee Type Description','Employee Status ID','Employee Status Description','Item Revision Date','Revision Number',
                   'Primary Location','Total Hours','Credit Hours','CPE Hours'], axis=1, inplace=True)

In [ ]:
data_success.TotalHours.sum()

2505.1899999999996

In [ ]:
len(data_success['Email'].unique())

1545

In [ ]:
test =data_success[~data_success['TotalHours'].isna()]
len(test['Email'].unique())

1287

In [ ]:
data_success.to_csv('successfactors.csv',index=False)

In [ ]:
data_success['LastViewDate']= pd.to_datetime(data_success['LastViewDate'])
data_success['LastViewDate'] = data_success.LastViewDate.dt.date

In [ ]:
#Creating a column that shows the different fiscal periods of 2022
d= {'1' : [('08-30-2021', '09-26-2021')],
  '2' : [('09-27-2021', '10-24-2021')],
  '3' : [('10-25-2021', '11-21-2021')],
  '4' : [('11-22-2021', '12-19-2021')],
  '5' : [('12-20-2021', '01-16-2022')],
  '6' : [('01-17-2022', '02-13-2022')],
  '7' : [('02-14-2022', '03-13-2022')],
  }

#What should I call the date
data_success['LastViewDate'] = pd.to_datetime(data_success['LastViewDate'], format='%Y/%m/%d')

for k, v in d.items():
    for s, e in v:
        data_success.loc[data_success['LastViewDate'].between(s,e), 'Period'] = k

### **Ad Hoc and CTT Data**

In [ ]:
ad_hoc = pd.read_csv('/content/Ad-hoc_P8.csv')

In [ ]:
#data cleaning


#filter only the data we want to capture, "yes" means data we need to capture
ad_hoc = ad_hoc.loc[ad_hoc['Include in adhoc report?']=='YES']
#removing columns that are not necessary in our analysis
ad_hoc.drop(['Timestamp','Your name:', 'Dept:', 'I report to:' , 'Captured', 'Notes','Include in adhoc report?'], axis=1, inplace=True)
#renaming columns to match similar column names in our master dataset
ad_hoc.rename(columns = {'Email Address' : 'Email', 'FY22 ' : 'Period','Date Completed' : 'LastViewDate',
                         'Total hours of instruction :' : 'TotalHours','Course Name:' : 'ContentName','Vendor that provided training:' : 'Platform'}, inplace= True)
#adding column to denote that the learning platform is "other"
ad_hoc.Platform = 'Other'

ad_hoc['LastViewDate']= pd.to_datetime(ad_hoc['LastViewDate'])
#this part is hard coded and needs to be changed per fiscal period
ad_hoc = ad_hoc[(ad_hoc['LastViewDate'] > '2022-03-13') & (ad_hoc['LastViewDate']<'2022-04-11')]

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
d= {'1' : [('2021-08-30', '2021-09-26')],
  '2' : [('2021-09-27', '2021-10-24')],
  '3' : [('2021-10-25', '2021-11-21')],
  '4' : [('2021-11-22', '2021-12-19')],
  '5' : [('2021-12-20', '2022-01-16')],
  '6' : [('2022-01-17', '2022-02-13')],
  '7' : [('2022-02-14', '2022-03-13')],
  '8' : [('2022-03-14', '2022-04-10')],
  }

#What should I call the date
ad_hoc['LastViewDate'] = pd.to_datetime(ad_hoc['LastViewDate'], format='%m/%d/%Y')

for k, v in d.items():
    for s, e in v:
        ad_hoc.loc[ad_hoc['LastViewDate'].between(s,e), 'Period'] = k

In [ ]:
len(ad_hoc.Email.unique())

70

### **Merging All Datasets Together **

In [ ]:
sap_hub.TotalHours.sum()

402.8563888888889

In [ ]:
#load old dataset
prev_data = pd.read_csv('/content/data_total_All.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (37) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
#changing the LastViewDate column in every data source to a date type object
content_usage['LastViewDate']= pd.to_datetime(content_usage['LastViewDate'])
linkedin_lmanage['LastViewDate']= pd.to_datetime(linkedin_lmanage['LastViewDate'])
microsoft['LastViewDate']= pd.to_datetime(microsoft['LastViewDate'])
data_success['LastViewDate']= pd.to_datetime(data_success['LastViewDate'])
sap_hub['LastViewDate']= pd.to_datetime(sap_hub['LastViewDate'])
ad_hoc['LastViewDate']= pd.to_datetime(ad_hoc['LastViewDate'])
content_usage['LastViewDate'] = content_usage.LastViewDate.dt.date
linkedin_lmanage['LastViewDate'] = linkedin_lmanage.LastViewDate.dt.date
microsoft['LastViewDate'] = microsoft.LastViewDate.dt.date
data_success['LastViewDate'] = data_success.LastViewDate.dt.date
sap_hub['LastViewDate'] = sap_hub.LastViewDate.dt.date
ad_hoc['LastViewDate'] = ad_hoc.LastViewDate.dt.date

In [ ]:
#appending all the datasets together
data_total = content_usage.append(linkedin_lmanage, ignore_index = True)
data_total = data_total.append(microsoft, ignore_index=True)
data_total = data_total.append(data_success, ignore_index=True)
data_total = data_total.append(sap_hub, ignore_index=True)
data_total = data_total.append(ad_hoc, ignore_index=True)
#removing the contractor data, which are ones with an email address that has an underscore
data_total = data_total.loc[~data_total['Email'].str.contains(r'_{1}',na=False)]
#removing rows with no data in TotalHours column
data_total = data_total[data_total['TotalHours'].notna()]

In [ ]:
#appending the new data to the old dataset to merge it all together
data_total = data_total.append(prev_data, ignore_index=True)

In [ ]:
#Saving the dataset to a csv file to upload to Power BI Report.
#REMEMBER: Need to rename the file each fiscal period because you need the previous fiscal period data to run all the code
data_total.to_csv('data_total_p8.csv',index=False)